# Generation of data tables with data required for creating input files for multiobjective dam placement study in Myanmar
### T. Janus
### Mui Ne, 28/12/2023

**NOTE:** Use IFC names throughout!!!

In [ ]:
from __future__ import annotations
from typing import List, Dict
import pathlib
from dataclasses import dataclass
import numpy as np
import geopandas as gpd
import pandas as pd

In [ ]:
@dataclass
class PywrIFCMapper:
    """ """
    name_map: pd.DataFrame
        
    @classmethod
    def from_file(cls, file_path: pathlib.Path) -> PywrIFCMapper:
        """ """
        return cls(name_map=pd.read_csv(file_path))
    
    @property
    def ifc_pywr_map(self) -> Dict[str, str]:
        """ """
        return self.name_map.set_index('ifc_name')['pywr_name'].to_dict()
    
    @property
    def pywr_ifc_map(self) -> Dict[str, str]:
        """ """
        return self.name_map.set_index('pywr_name')['ifc_name'].to_dict()
        
    def print_cls(self) -> None:
        print(self.name_map)

In [ ]:
col_name_map_reemission_inputs = {
    "res_area_fractions_2": "urban_area_fraction_1",
    "res_area_fractions_5": "crop_area_fraction_1",
    "res_area_fractions_7": "forest_area_fraction_1",
    "res_area_fractions_11": "urban_area_fraction_2",
    "res_area_fractions_14": "crop_area_fraction_2",
    "res_area_fractions_16": "forest_area_fraction_2",
    "res_area_fractions_20": "urban_area_fraction_3",
    "res_area_fractions_23": "crop_area_fraction_3",
    "res_area_fractions_25": "forest_area_fraction_3",
}

## 1. Import the required datasets

In [ ]:
# Import tabular heet output data, pywr output hp summary data and shp data (for spatial inference)
reservoirs_shp_data_path = pathlib.Path("bin/heet_outputs_MIN_LOW_PRI/reservoirs_updated.shp")
hp_summary_data_path = pathlib.Path("intermediate/merged_table.xlsx")
reemission_output_data_path = pathlib.Path("outputs/reemission/combined/combined_outputs.csv")
ifc_pywr_map_path = pathlib.Path("config/ifc_pywr_name_map.csv")
ifc_db_shp_path = pathlib.Path("bin/gis_layers/ifc_database/all_dams_replaced_refactored.shp")

pywr_ifc_mapper = PywrIFCMapper.from_file(ifc_pywr_map_path)

# 0. IFC database of dams
ifc_gdf = gpd.read_file(ifc_db_shp_path)\
    .loc[:,['IFC_ID', 'geometry']]\
    .rename(columns={'IFC_ID': 'ifc_id'})

# 1. Import spatial data and remove all reservoirs that are not HP or multipurpose
reservoirs_gdf = gpd.read_file(reservoirs_shp_data_path)\
    .query("type == 'hydroelectric' | type == 'multipurpose'")\
    .loc[:,['id', 'r_area_km2', 'geometry']]\
    .rename(columns={"id": "ifc_id"})

reservoirs_gdf_all = gpd.read_file(reservoirs_shp_data_path)\
    .query("type == 'hydroelectric' | type == 'multipurpose'")\
    .rename(columns={"id": "ifc_id"})

# 2. Import tabular data with emissions and hp production
hp_summary = pd.read_excel(hp_summary_data_path)\
    .loc[:,[
        'ifc_id', 'dam_name', 'res_area', 'status_2_ifc', 'ro_r_or_sto_ifc', 'mean', 'pctile_10', 
        'co2_total_per_year', 'ch4_total_per_year', 'hp_type_reem']]
hp_summary['status'] = np.where(hp_summary['status_2_ifc'].isin(['E']), 'Existing', 'Future')
hp_summary = hp_summary\
    .assign(**{col: hp_summary[col].fillna(0) for col in 
               ['co2_total_per_year', 'ch4_total_per_year', 'res_area']})\
    .drop_duplicates()\
    .assign(dam_name=hp_summary['dam_name'].replace(pywr_ifc_mapper.pywr_ifc_map))\
    .sort_values(by="dam_name", ascending=True)\
    .eval('tot_em = co2_total_per_year + ch4_total_per_year')\
    .drop(columns=['co2_total_per_year', 'ch4_total_per_year', 'status_2_ifc', 'res_area'])\
    .rename(columns={
        "mean": "HP_mean", "pctile_10": "HP_firm", 'ro_r_or_sto_ifc' : 'hp_type_ifc',
        "dam_name": "name"})

# 3. Import the inputs tab from the outputs file from re-emission
reemission_inputs = pd.read_csv(reemission_output_data_path)\
    .loc[:,[
        'id', 'type', 'Scenario', 'res_area_fractions_2', "res_area_fractions_5",
        "res_area_fractions_7", "res_area_fractions_11", "res_area_fractions_14", "res_area_fractions_16",
        "res_area_fractions_20", "res_area_fractions_23", "res_area_fractions_25"]]\
    .query("type != 'irrigation' & Scenario == 'MIN_LOW_PRIM'")\
    .rename(columns=col_name_map_reemission_inputs)\
    .rename(columns={'id': 'ifc_id'})\
    .eval('urban_area_fraction = urban_area_fraction_1 + urban_area_fraction_2 + urban_area_fraction_3')\
    .eval('forest_area_fraction = forest_area_fraction_1 + forest_area_fraction_2 + forest_area_fraction_3')\
    .eval('crop_area_fraction = crop_area_fraction_1 + crop_area_fraction_2 + crop_area_fraction_3')\
    .drop(columns=[
        "Scenario", "urban_area_fraction_1", "urban_area_fraction_2", "urban_area_fraction_3",
        "crop_area_fraction_1", "crop_area_fraction_2", "crop_area_fraction_3",
        "forest_area_fraction_1", "forest_area_fraction_2", "forest_area_fraction_3"])

In [ ]:
reservoirs_gdf_all.head()

## 2. Join all three dataframes together

In [ ]:
merged_df = reemission_inputs\
    .merge(hp_summary, on='ifc_id', how='right')\
    .merge(reservoirs_gdf, on='ifc_id', how='left')

merged_df = merged_df\
    .assign(**{col: merged_df[col].fillna(0) for col in 
               ['urban_area_fraction', 'forest_area_fraction', 'crop_area_fraction',
                'r_area_km2']})\
    .merge(ifc_gdf, on='ifc_id', suffixes=('_df1', '_ifc'))


merged_df = merged_df\
    .assign(**{col: merged_df[col].fillna(merged_df['geometry_ifc']) for col in 
               ['geometry_df1']})\
    .assign(**{col: merged_df[col].fillna('hydroelectric') for col in 
               ['type']})\
    .drop(columns='geometry_ifc')\
    .rename(columns={'geometry_df1': 'geometry'})\
    .eval('urban_area_loss_km2 = urban_area_fraction * r_area_km2')\
    .eval('forest_area_loss_km2 = forest_area_fraction * r_area_km2')\
    .eval('crop_area_loss_km2 = crop_area_fraction * r_area_km2')\
    .drop(columns=['urban_area_fraction', 'forest_area_fraction', 'crop_area_fraction', 'r_area_km2'])

In [ ]:
# Join dataframes for plotting maps (visualising MOO results)
merged_df_plot = reemission_inputs\
    .merge(hp_summary, on='ifc_id', how='right')\
    .merge(reservoirs_gdf_all, on='ifc_id', how='left')
_
merged_df_plot = merged_df_plot\
    .assign(**{col: merged_df_plot[col].fillna(0) for col in 
               ['urban_area_fraction', 'forest_area_fraction', 'crop_area_fraction',
                'r_area_km2']})\
    .merge(ifc_gdf, on='ifc_id', suffixes=('_df1', '_ifc'))\
    .drop(columns = ["type_x", "urban_area_fraction","c_biome", "c_soil_typ", "c_mean_slo", "n_populati",
                    "r_msocs_kg", "r_mghr_all", "r_mghr_may", "r_mghr_nov", "r_mean_ann", "c_mar_mm",
                    "c_map_mm", "c_mpet_mm", "c_masm_mm", "c_area_km2", "c_mean_ols",
                    "ms_length", "tot_em_y", "geometry_df1"])
# Calculate composite values for plotting
merged_df_plot["coordinates_1"] = merged_df_plot.geometry_ifc.apply(lambda p: p.x)
merged_df_plot["coordinates_0"] = merged_df_plot.geometry_ifc.apply(lambda p: p.y)
merged_df_plot['HP Production [GWh/year]'] = merged_df_plot["HP_mean"] * 365.25 * 24 / 1_000
merged_df_plot['Mean HP [GWh/d]'] = merged_df_plot["HP_mean"] * 24 / 1_000
merged_df_plot['Firm HP [GWh/d]'] = merged_df_plot['HP_firm'] * 24 / 1_000
merged_df_plot['Firm Power Ratio, [%]'] = merged_df_plot['Firm HP [GWh/d]'] / merged_df_plot['Mean HP [GWh/d]'] * 100
# Fill tot_em values for RoR HP
# Force emission intensity of 3 gCO2eq/kWh
ror_em_intensity = 3
merged_df_plot.loc[merged_df_plot['tot_em_x'] == 0, 'tot_em_x'] = \
    merged_df_plot['HP_mean'] * ror_em_intensity / 1_000 * 365.25 * 24
merged_df_plot['GHG intensity [gCO2,eq/kWh]'] = \
    merged_df_plot['tot_em_x'] / merged_df_plot['HP_mean'] * 1_000 / 365.25 / 24
merged_df_plot['Volume, Mm3'] = merged_df_plot['r_volume_m'] / 1_000_000
merged_df_plot['Area, km2'] = merged_df_plot['r_area_km2']
# Note: tot_em is in tCO2eq yr-1
merged_df_plot['GHG, tCO2eq/yr'] = merged_df_plot['tot_em_x']
merged_df_plot = merged_df_plot\
    .drop(columns=['tot_em_x', 'r_volume_m', 'r_area_km2'])
# Fill NA volume values with default_ror_volume
default_ror_volume = 5 # Mm3
merged_df_plot['Volume, Mm3'] = merged_df_plot['Volume, Mm3'].fillna(default_ror_volume)
merged_df_plot.to_csv(pathlib.Path("intermediate/dams_for_plotting_moo.csv"), index=False)

In [ ]:
# Specify the desired column order
col_order = [
    'ifc_id', 'name', 'type', 'status', 'hp_type_ifc', 'hp_type_reem',
    'HP_mean', 'HP_firm', 'tot_em', 'urban_area_loss_km2', 'forest_area_loss_km2',
    'crop_area_loss_km2', 'geometry']

assert set(merged_df.columns) == set(col_order)

# Reorder columns based on the desired order
merged_df = merged_df[col_order]
merged_gdf = gpd.GeoDataFrame(merged_df, geometry=merged_df['geometry'])

In [ ]:
merged_gdf.head()

## 3. Intersect the merged dataframe with village data points

In [ ]:
villages_gdf = gpd.read_file(
    pathlib.Path(
        "bin/gis_layers/hotosm_mmr_populated_places_points_shp/hotosm_mmr_populated_places_points.shp"))

In [ ]:
flooded_villages = \
    gpd.sjoin(villages_gdf, merged_gdf, how='left', op='within')\
    .groupby('name_right').count()['osm_id'].reset_index()\
    .rename(columns={"osm_id" : "count", "name_right" : "name"})

flooded_villages

## 4. Add information about flooded villages to `merged_gdf`

In [ ]:
final_gdf = merged_gdf.merge(flooded_villages, on="name", how='left')\
    .rename(columns={"count" : "flooded_villages"})
final_gdf["flooded_villages"] = final_gdf["flooded_villages"].fillna(0)
final_gdf.head()

## 5. Save dataframes for post-processing
1. Save Future reservoirs for optimization purposes
2. Save Existing reservoirs for statistics on existing hydroelectric reservoirs

In [ ]:
output_folder = pathlib.Path("outputs/moo")
if not output_folder.exists():
    output_folder.mkdir()
# Add status_int column
final_gdf['status_int'] = final_gdf['status'].map({'Existing': 1, 'Future': 0})
final_gdf\
    .drop(columns=['geometry'])\
    .to_csv(output_folder/'all_hp.csv')
    
final_gdf\
    .query('status == "Existing"')\
    .drop(columns=['geometry'])\
    .to_csv(output_folder/'existing_hp.csv')

final_gdf\
    .query('status == "Future"')\
    .drop(columns=['geometry', 'type', 'status', 'hp_type_ifc', 'hp_type_reem'])\
    .to_csv(output_folder/'future_hp.csv')